通过`send()`函数可以实现`next()`函数功能，即继续执行协程直到被下一个`yield`语句挂起，同时实现向协程传递参数功能。

协程的四种状态，可以通过`inspect.getgeneratorstate()`获得：

- GEN_CREATED：已创建，等待启动
- GEN_RUNNING：正在执行
- GEN_SUSPENDED：由yield挂起
- GEN_CLOSE：执行结束

协程最初处于GEN_CREATED状态，必须通过调用一次`next()`或`send(None)`才会开始启动，只有在协程处于GEN_SUSPENDED状态时才能通过`send()`向其传递参数，否则会抛出异常。

启动协程的过程被称为协程初始化。通过`coroutil.coroutine`装饰器可以实现自动初始化的功能。

## 从协程返回值

### 通过return语句返回结果

现在，协程可以通过return语句返回一个结果。return语句会抛出一个StopIteration异常，其value属性即为返回值。

In [ ]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

### 通过yield from语句返回结果

`yield from subgen()`表达式能够将当前协程挂起，同时启动一个子协程，直到子协程返回结果才继续执行。

In [1]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

list(gen())

['A', 'B', 1, 2]

In [2]:
def gen():
    yield from 'AB'
    yield from range(1, 3)

list(gen())

['A', 'B', 1, 2]

In [ ]:
def chain(*iterables):
    for it in iterables:
        yield from it

s = 'ABC'
t = tuple(range(3))
list(chain())

`yield from x`表达式首先会调用`iter(x)`获取一个迭代器。

`yield from`用于为外部调用者与内部子生成器提供一个双向管道，实现了两者间的数据互相传递，同时不需要对抛出的异常进行额外的处理。

In [3]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

# the subgenerator
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

def grouper(results, key):
    while True:
        results[key] = yield from averager()
        
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None) # important!
        
    # print(results) # uncomment to debug
    report(results)
    
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
            result.count, group, result.average, unit))

data = {
    'girls;kg': [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m': [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


1. 调用者：用于调用委托生成器
2. 委托生成器：包含`yield from`表达式的生成器函数
3. 子生成器：位于`yield from`表达式之后

`yield from`表达式的六个性质：

1. 子生成器的`yield`表达式返回的值将直接传递给委托生成器的调用者。
2. 任何通过`send()`函数传递给委托生成器的值都会直接传递给子生成器，同时调用子生成器的`next()`函数。
3. 生成器中的`return expr`语句会抛出`StopIteration(expr)`异常并退出生成器。
4. `yield from`表达式返回的值时子生成器终止时抛出`StopIteration`异常的第一个参数。
5. // TODO
6. // TODO